In [47]:
import os

cidade = 'itabira'
caminho_arquivos_cidade = os.getcwd() + '\\base-dados\\' + cidade
caminho_medidas_refinadas = caminho_arquivos_cidade + '\\' + cidade + '_medidas_refinadas.csv'
caminho_medidas = caminho_arquivos_cidade + '\\' + cidade + '_medidas.csv'
caminho_grafo = caminho_arquivos_cidade + '\\' + cidade + '.graphml'
caminho_dados_policia = os.getcwd() + '\\dados_cidades_projeto\\DadosRefinados_' + cidade + '.csv'

In [48]:
import osmnx as ox
import networkx as nx

#G = ox.load_graphml(caminho_arquivos_cidade + '\\' + cidade + '.graphml')
#G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')

G = ox.load_graphml(caminho_grafo)
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')

#dc = nx.degree(G)
#node = G.nodes[5]
#node

result = ox.stats.streets_per_node_counts(G)
result

# cores_ocorrencias = ox.plot.get_node_colors_by_attr(G, "street_count", cmap="rainbow")

# for u, v, data in G.edges.data():
#     data.clear()

# ox.plot.plot_graph(
#    G, bgcolor="k", node_color=cores_ocorrencias, node_size=5, edge_linewidth=1, edge_color="#ffffee"
# )

{0: 0, 1: 337, 2: 17, 3: 2151, 4: 396, 5: 7}

In [2]:
#calculando a métrica da estrutura urbana de cada cidade e armazendando em csv
# GRAFO MÉTRICAS

import networkx as nx
import pandas as pd
import osmnx as ox

G = ox.load_graphml(caminho_grafo)
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')
D = ox.utils_graph.get_digraph(G)

bc = nx.betweenness_centrality(D)
cc = nx.closeness_centrality(D)
dc = nx.degree_centrality(D)
ec = nx.eigenvector_centrality(D, max_iter=600)
pc = nx.pagerank(D, max_iter=600)

cols = ['closeness_centrality']
df = pd.DataFrame.from_dict(cc, orient='index', columns=cols)
df['betweenes_centrality'] = bc.values()
df['degree_centrality'] = dc.values()
df['eigenvector_centrality'] = ec.values()
df['pagerank_centrality'] = pc.values()

caminho_medidas = caminho_arquivos_cidade + '\\' + cidade + '_medidas.csv'
df.to_csv(caminho_medidas)

In [3]:
#registrando latitude e longitude de cada cruzamento na planilha
import osmnx as ox
import networkx as nx
import pandas as pd

G = ox.load_graphml(caminho_grafo)
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')

pontos = {i: 0 for i in range(nx.number_of_nodes(G))}

for u, data in G.nodes(data=True):
    local_cruzamento = (data['y'], data['x'])
    pontos[u] = local_cruzamento

dadoscalculados = pd.read_csv(caminho_medidas)
dadoscalculados['latitude e longitude'] = pontos.values()
dadoscalculados.to_csv(caminho_medidas, index=False);

In [4]:
#calculando a quantidade de ocorrências registradas em cada cruzamento da cidade
# GRAFO OCORRENCIAS

import osmnx as ox
import networkx as nx
import pandas as pd
import haversine as hs

G = ox.load_graphml(caminho_arquivos_cidade + '\\' + cidade + '.graphml')
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')

df = pd.read_csv(caminho_dados_policia)
dadoscalculados = pd.read_csv(caminho_medidas)
nr_nodes = nx.number_of_nodes(G)

ocorrencias = {i: 0 for i in range(nr_nodes)}

for index, row in df.iterrows():
    max_dist = float('inf')
    cruzamento_selecionado = None;
    local_ocorrencia = (row['Latitude'], row['Longitude'])
    
    for u, data in G.nodes(data=True):
        local_cruzamento = (data['y'], data['x'])
        dist = hs.haversine(local_ocorrencia, local_cruzamento, unit=hs.Unit.METERS)
        if(dist < max_dist):
            max_dist = dist
            cruzamento_selecionado = u
    
    if cruzamento_selecionado is not None:
        ocorrencias[cruzamento_selecionado] += 1

dadoscalculados['quantidade_ocorrencia'] = ocorrencias.values()
dadoscalculados.to_csv(caminho_medidas, index=False);

In [5]:
#calculo da distancia para a universidade federal

import osmnx as ox
import networkx as nx
import pandas as pd
import haversine as hs

G = ox.load_graphml(caminho_grafo)
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')

localizacao_universidades = {
                            'alfenas': (-21.420734139812197, -45.94857609208687),
                            'itabira':(-19.673061652058852, -43.212421534406204),
                            'itajuba':(-22.421057250613636, -45.460112158690066),
                            'joaomonlevade': (-19.8359317141204, -43.16770629585404),                            
                            'lavras': (-21.230714620911264, -44.994126378125785),
                            'ouropreto': (-20.395772569014458, -43.51068671213827),
                            'saojoao': (-21.13959592057899, -44.26047341001594),
                            'vicosa': (-20.760802504862596, -42.870214344859825)
                            }

localizacao_universidade = localizacao_universidades[cidade]
distancias = {i: 0 for i in range(nx.number_of_nodes(G))}

for u, data in G.nodes(data=True):
    local_cruzamento = (data['y'], data['x'])
    dist = hs.haversine(localizacao_universidade, local_cruzamento, unit=hs.Unit.METERS)
    distancias[u] = dist

dadoscalculados = pd.read_csv(caminho_medidas)
dadoscalculados['distancia_universidade'] = distancias.values()
dadoscalculados.to_csv(caminho_medidas, index=False);

In [6]:
#calculando a quantidade de ocorrências registradas em cada cruzamento da cidade - utilizando a segunda estratégia
# GRAFO OCORRENCIAS - segunda estratégia

import osmnx as ox
import networkx as nx
import pandas as pd
import haversine as hs

G = ox.load_graphml(caminho_arquivos_cidade + '\\' + cidade + '.graphml')
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')

df = pd.read_csv(caminho_dados_policia)
dadoscalculados = pd.read_csv(caminho_medidas)
nr_nodes = nx.number_of_nodes(G)

ocorrencias = {i: 0 for i in range(nr_nodes)}

for index, row in df.iterrows():
    max_dist_1 = float('inf')
    max_dist_2 = float('inf')
    max_dist_3 = float('inf')
    max_dist_4 = float('inf')

    cruzamento_selecionado_1 = None;
    cruzamento_selecionado_2 = None;
    cruzamento_selecionado_3 = None;
    cruzamento_selecionado_4 = None;
    local_ocorrencia = (row['Latitude'], row['Longitude'])
    
    for u, data in G.nodes(data=True):
        local_cruzamento = (data['y'], data['x'])
        dist = hs.haversine(local_ocorrencia, local_cruzamento, unit=hs.Unit.METERS)
        
        if(dist < max_dist_1):
            max_dist_4 = max_dist_3
            cruzamento_selecionado_4 = cruzamento_selecionado_3
            
            max_dist_3 = max_dist_2
            cruzamento_selecionado_3 = cruzamento_selecionado_2
            
            max_dist_2 = max_dist_1
            cruzamento_selecionado_2 = cruzamento_selecionado_1
            
            max_dist_1 = dist
            cruzamento_selecionado_1 = u
            
        elif(dist < max_dist_2):
            max_dist_4 = max_dist_3
            cruzamento_selecionado_4 = cruzamento_selecionado_3
            
            max_dist_3 = max_dist_2
            cruzamento_selecionado_3 = cruzamento_selecionado_2
            
            max_dist_2 = dist
            cruzamento_selecionado_2 = u
            
        elif(dist < max_dist_3):
            max_dist_4 = max_dist_3
            cruzamento_selecionado_4 = cruzamento_selecionado_3
            
            max_dist_3 = dist
            cruzamento_selecionado_3 = u
        
        elif(dist < max_dist_4):
            max_dist_4 = dist
            cruzamento_selecionado_4 = u
    
    if cruzamento_selecionado_1 is not None:
        ocorrencias[cruzamento_selecionado_1] += ((4/4)/2.5)
    
    if cruzamento_selecionado_2 is not None:
        ocorrencias[cruzamento_selecionado_2] += ((3/4)/2.5)
        
    if cruzamento_selecionado_3 is not None:
        ocorrencias[cruzamento_selecionado_3] += ((2/4)/2.5)
        
    if cruzamento_selecionado_4 is not None:
        ocorrencias[cruzamento_selecionado_4] += ((1/4)/2.5)

dadoscalculados['quantidade_ocorrencia_2'] = ocorrencias.values()
dadoscalculados.to_csv(caminho_medidas, index=False);

In [7]:
#calculando a quantidade de ocorrências registradas em cada cruzamento da cidade - utilizando a segunda estratégia
# GRAFO OCORRENCIAS - segunda estratégia

import osmnx as ox
import networkx as nx
import pandas as pd
import haversine as hs

G = ox.load_graphml(caminho_arquivos_cidade + '\\' + cidade + '.graphml')
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')

df = pd.read_csv(caminho_dados_policia)
dadoscalculados = pd.read_csv(caminho_medidas)
nr_nodes = nx.number_of_nodes(G)

ocorrencias = {i: 0 for i in range(nr_nodes)}

for index, row in df.iterrows():
    max_dist_1 = float('inf')
    max_dist_2 = float('inf')
    max_dist_3 = float('inf')
    max_dist_4 = float('inf')

    cruzamento_selecionado_1 = None;
    cruzamento_selecionado_2 = None;
    cruzamento_selecionado_3 = None;
    cruzamento_selecionado_4 = None;
    local_ocorrencia = (row['Latitude'], row['Longitude'])
    
    for u, data in G.nodes(data=True):
        local_cruzamento = (data['y'], data['x'])
        dist = hs.haversine(local_ocorrencia, local_cruzamento, unit=hs.Unit.METERS)
        
        if(dist < max_dist_1):
            max_dist_4 = max_dist_3
            cruzamento_selecionado_4 = cruzamento_selecionado_3
            
            max_dist_3 = max_dist_2
            cruzamento_selecionado_3 = cruzamento_selecionado_2
            
            max_dist_2 = max_dist_1
            cruzamento_selecionado_2 = cruzamento_selecionado_1
            
            max_dist_1 = dist
            cruzamento_selecionado_1 = u
            
        elif(dist < max_dist_2):
            max_dist_4 = max_dist_3
            cruzamento_selecionado_4 = cruzamento_selecionado_3
            
            max_dist_3 = max_dist_2
            cruzamento_selecionado_3 = cruzamento_selecionado_2
            
            max_dist_2 = dist
            cruzamento_selecionado_2 = u
            
        elif(dist < max_dist_3):
            max_dist_4 = max_dist_3
            cruzamento_selecionado_4 = cruzamento_selecionado_3
            
            max_dist_3 = dist
            cruzamento_selecionado_3 = u
        
        elif(dist < max_dist_4):
            max_dist_4 = dist
            cruzamento_selecionado_4 = u
    
    #divide 1000 (1km) by the dist measured
    if cruzamento_selecionado_1 is not None:
        ocorrencias[cruzamento_selecionado_1] += (1000/max_dist_1)
    
    if cruzamento_selecionado_2 is not None:
        ocorrencias[cruzamento_selecionado_2] += (1000/max_dist_2)
        
    if cruzamento_selecionado_3 is not None:
        ocorrencias[cruzamento_selecionado_3] += (1000/max_dist_3)
        
    if cruzamento_selecionado_4 is not None:
        ocorrencias[cruzamento_selecionado_4] += (1000/max_dist_4)

dadoscalculados['medida_influencia_crime'] = ocorrencias.values()
dadoscalculados.to_csv(caminho_medidas, index=False);

In [18]:
import osmnx as ox
import networkx as nx

#G = ox.load_graphml(caminho_arquivos_cidade + '\\' + cidade + '.graphml')
#G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')

G = ox.load_graphml(caminho_grafo)
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='old_index')

#dc = nx.degree(G)
#node = G.nodes[5]
#node

result = ox.stats.streets_per_node_counts(G)
result

# cores_ocorrencias = ox.plot.get_node_colors_by_attr(G, "street_count", cmap="rainbow")

# for u, v, data in G.edges.data():
#     data.clear()

# ox.plot.plot_graph(
#    G, bgcolor="k", node_color=cores_ocorrencias, node_size=5, edge_linewidth=1, edge_color="#ffffee"
# )

{0: 0, 1: 337, 2: 17, 3: 2151, 4: 396, 5: 7}